In [1]:
from bs4 import BeautifulSoup
import selenium
import requests

In [19]:
df = pd.read_csv("./input_v2.txt",sep="/", names=["row"], encoding="utf8").dropna()

In [20]:
def transform_row(row):
    # row = row.encode("utf-8")
    row = re.sub(r"^[0-9\.]+", "", row)
    row = re.sub(r"^[\.,\?]+", "", row)
    
    row = row.strip()
    return row 

df["row"] = df.row.apply(transform_row)
df.head(10)

,row
0,"Trăm năm trong cõi người ta,"
1,Chữ tài chữ mệnh khéo là ghét nhau.
2,"Trải qua một cuộc bể dâu,"
3,Những điều trông thấy mà đau đớn lòng.
4,"Lạ gì bỉ sắc tư phong,"
5,Trời xanh quen thói má hồng đánh ghen.
6,"Cảo thơm lần giở trước đèn,"
7,Phong tình có lục còn truyền sử xanh.
8,"Rằng năm Gia Tĩnh triều Minh,"
9,"Bốn phương phẳng lặng, hai kinh vững vàng."


In [22]:
output_file = "truyen_kieu_clean.txt"
with open(output_file, "w", encoding='utf-8') as f:
    for row in df["row"]:
        f.write(row + '\n')
    f.close()

print("Successfully save clean version of Truyen Kieu")

Successfully save clean version of Truyen Kieu
